# Import libraries

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.models import vgg16, VGG16_Weights
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
from PIL import Image
import os
import numpy as np

# Database creations using pytorch Dataset 

In [4]:
class ImageAuthenticityDataset(Dataset):
    """Dataset for image quality assessment."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the CSV file with annotations.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """Returns the number of samples in the dataset."""
        return len(self.data)

    def __getitem__(self, idx):
        """
        Retrieves an image and its labels by index.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            tuple: A tuple (image, labels) where:
                image (PIL.Image): The image.
                labels (torch.Tensor): Tensor containing quality and authenticity scores.
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(os.getcwd(), self.data.iloc[idx, 3])  # image_path column
        image = Image.open(img_name).convert('RGB')
        authenticity = self.data.iloc[idx, 1]  # Authenticity column
        labels = torch.tensor([authenticity], dtype=torch.float)


        if self.transform:
            image = self.transform(image)

        return image, labels


# Definitions of the models

In [5]:
class AuthenticityPredictor(nn.Module):
    def __init__(self, freeze_backbone=True):
        super().__init__()
        # Load pre-trained VGG16
        vgg = vgg16(weights=VGG16_Weights.DEFAULT)
        
        # Freeze backbone if requested
        if freeze_backbone:
            for param in vgg.features.parameters():
                param.requires_grad = False
                
        # Extract features up to fc2
        self.features = vgg.features
        self.avgpool = vgg.avgpool
        self.fc1 = vgg.classifier[:-1]  # Up to fc2 (4096 -> 128)
        
        # New regression head
        self.regression_head = nn.Sequential(
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)  # Predict authenticity
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        features = self.fc1(x)
        predictions = self.regression_head(features)
        return predictions, features

# Utility functions for training and evaluation

In [6]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10, device='cuda'):
    """
    Trains the model.

    Args:
        model (nn.Module): The model to train.
        dataloaders (dict): A dictionary containing the training and validation data loaders.
        criterion (nn.Module): The loss function.
        optimizer (optim.Optimizer): The optimizer.
        num_epochs (int): Number of epochs to train for. Defaults to 10.
        device (str): Device to use for training ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        nn.Module: The trained model.
    """
    model.to(device)
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:  # Iterate over training and validation phases
            print(f'{phase} phase')
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            for inputs, labels in dataloaders[phase]:  # Iterate over data in the current phase
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):  # Enable gradients only during training
                    outputs, _ = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)

            print(f'{phase} Loss: {epoch_loss:.4f}') # Print loss for the current phase

    print("Finished Training")
    return model

def test_model(model, dataloader, criterion, device='cuda'):

    """
    Tests the model on the test dataset.

    Args:
        model (nn.Module): The trained model.
        dataloader (DataLoader): The test data loader.
        criterion (nn.Module): The loss function.
        device (str): Device to use for testing ('cuda' or 'cpu'). Defaults to 'cuda'.

    Returns:
        float: The average loss on the test dataset.
    """
    model.eval()  # Set the model to evaluation mode
    model.to(device)
    running_loss = 0.0

    with torch.no_grad():  # Disable gradient calculation
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs, _ = model(inputs)
            
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

    test_loss = running_loss / len(dataloader.dataset)
    print(f'Test Loss: {test_loss:.4f}')
    return test_loss

def get_predictions(model, dataloader, device)-> tuple[torch.Tensor, torch.Tensor]:
    """
    Get predictions from the model.

    Args:
        model (nn.Module): The trained model.
        dataloader (DataLoader): The data loader.

    Returns:
        tuple: A tuple (predictions, labels) where:
            predictions (torch.Tensor): Predictions from the model.
            labels (torch.Tensor): Ground truth labels.
    """
    model.eval()  # Set the model to evaluation mode
    model.to(device)
    predictions = []
    labels = []

    with torch.no_grad():  # Disable gradient calculation
        for inputs, target in dataloader:
            outputs, _ = model(inputs.to(device))
            predictions.append(outputs)
            labels.append(target)

    #move to cpu and concatenate
    predictions = torch.cat(predictions).cpu()
    labels = torch.cat(labels).cpu()

    return predictions, labels

def get_regression_errors(tuple: tuple[torch.Tensor, torch.Tensor]) -> tuple[torch.Tensor, torch.Tensor]:
    """
    Get regression errors.

    Args:
        tuple: A tuple (predictions, labels) where:
            predictions (torch.Tensor): Predictions from the model.
            labels (torch.Tensor): Ground truth labels.

    Returns:
        tuple: A tuple (quality_errors, authenticity_errors) where:
            quality_errors (torch.Tensor): Quality errors.
            authenticity_errors (torch.Tensor): Authenticity errors.
    """
    predictions, labels = tuple
    authenticity_errors = predictions[:, 0] - labels[:, 0]
    return authenticity_errors

def get_rmse(errors: torch.Tensor) -> torch.Tensor:
    """
    Get the root mean squared error.

    Args:
        errors (torch.Tensor): Errors.

    Returns:
        torch.Tensor: Root mean squared error.
    """
    return torch.sqrt(torch.mean(errors ** 2))


## Training section

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data transformations for the ImageNet dataset
data_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

annotations_file = 'Dataset/AIGCIQA2023/mos_data.csv'

# Create the dataset
dataset = ImageAuthenticityDataset(csv_file=annotations_file, transform=data_transforms)

# Set random seeds for reproducibility
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])


# Create data loaders
BATCH_SIZE = 64
EPOCHS = 20
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# Create a dictionary containing the data loaders
dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader,
    'test': test_dataloader
}

model = AuthenticityPredictor()
criterion = nn.MSELoss()  # Mean Squared Error Loss (regression)
optimizer = optim.Adam(model.regression_head.parameters(), lr=0.001)

model_path = 'Models/VGG-16_authenticity_finetuned.pth'


## Train 

In [ ]:
authenticity_predictor_trained= train_model(model, dataloaders, criterion, optimizer, EPOCHS, device)

# Save the trained model
torch.save(authenticity_predictor_trained.state_dict(), "Models/VGG-16_authenticity_finetuned.pth")


Epoch 0/19
----------
train phase
train Loss: 777.7876
val phase
val Loss: 134.2582
Epoch 1/19
----------
train phase
train Loss: 206.4005
val phase
val Loss: 196.0453
Epoch 2/19
----------
train phase
train Loss: 161.9668
val phase
val Loss: 193.4858
Epoch 3/19
----------
train phase
train Loss: 149.4155
val phase
val Loss: 183.4053
Epoch 4/19
----------
train phase
train Loss: 146.0410
val phase
val Loss: 156.5203
Epoch 5/19
----------
train phase
train Loss: 141.8292
val phase
val Loss: 167.9337
Epoch 6/19
----------
train phase
train Loss: 131.4791
val phase
val Loss: 165.7670
Epoch 7/19
----------
train phase
train Loss: 133.6336
val phase
val Loss: 164.5004
Epoch 8/19
----------
train phase
train Loss: 138.0414
val phase
val Loss: 160.4423
Epoch 9/19
----------
train phase
train Loss: 128.6270
val phase
val Loss: 158.1449
Epoch 10/19
----------
train phase
train Loss: 130.6597
val phase
val Loss: 149.2516
Epoch 11/19
----------
train phase
train Loss: 123.8050
val phase
val Loss:

## Utilities for pruning 

In [ ]:
def compute_feature_map_importance(model, dataloader, device, layer_name) -> tuple[np.ndarray, np.ndarray]:
    """Computes the importance of each feature map in a convolution
    layer by measuring the change in predictions when the feature map is zero
    out.
    
    Returns:
        tuple: (indices, importance_scores) where both are numpy arrays
    """
    #if importance_scores.npy exists, load it
    if os.path.exists('Ranking_arrays/authenticity_importance_scores.npy'):
        print("Importance scores already computed, loading from file")
        return np.load('Ranking_arrays/authenticity_importance_scores.npy')
    
    model.eval()
    model.to(device)
    importance_scores = []
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    baseline_predictions = get_predictions(model, dataloader, device)
    regression_errors = get_regression_errors(baseline_predictions)
    baseline_authenticity_rmse = get_rmse(regression_errors)

    
    print(f'Authenticity baseline RMSE: {baseline_authenticity_rmse:.4f}')

    with torch.no_grad():
        for i in range(layer.out_channels):
            # Create a backup of the weights and bias
            backup_weights = layer.weight[i, ...].clone()
            backup_bias = layer.bias[i].clone() if layer.bias is not None else None

            # Zero out the i-th output channel
            layer.weight[i, ...] = 0
            if layer.bias is not None:
                layer.bias[i] = 0

            # Get predictions with the pruned feature map
            pruned_predictions = get_predictions(model, dataloader, device)
            pruned_regression_errors = get_regression_errors(pruned_predictions)
            pruned_authenticity_rmse = get_rmse(pruned_regression_errors)
            
    
            # Compute importance score
            importance_score = baseline_authenticity_rmse - pruned_authenticity_rmse
            importance_scores.append([i, importance_score])
            

            print(f'Feature map {i}: Importance score: {importance_score:.4f}')
            
            # After computing importance, restore weights and bias
            layer.weight[i, ...] = backup_weights
            if layer.bias is not None:
                layer.bias[i] = backup_bias 

    sorted_importance_scores = sorted(importance_scores, key=lambda x: x[1], reverse=True)
    # save np array 
    np.save('authenticity_importance_scores.np', sorted_importance_scores)
    return np.array(sorted_importance_scores)

def remove_noisy_feature_maps(model, dataloader, device, layer_name, sorted_importance_scores, model_path='Models/pruned_model.pth'):
    """
    Remove noisy feature maps from a convolutional layer based on importance scores.
    Feature maps are zeroed out one by one and kept zeroed only if model performance improves.
    
    Args:
        model: The neural network model
        dataloader: DataLoader for evaluation
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        sorted_importance_scores: List of tuples (channel_index, importance_score) sorted by importance
        
    Returns:
        Dictionary with pruning results and performance metrics
    """
    model.eval()
    model.to(device)
    
    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    
    # Create a backup of the original weights and bias
    original_weights = layer.weight.clone()
    original_bias = layer.bias.clone() if layer.bias is not None else None
    
    # Initialize tracking variables
    removed_features = []
    rmse_history = []
    
    # Get baseline performance
    baseline_predictions = get_predictions(model, dataloader, device)
    baseline_regression_errors = get_regression_errors(baseline_predictions)
    baseline_authenticity_rmse = get_rmse(baseline_regression_errors)
    
    
    print(f"Baseline authenticity RMSE: {baseline_authenticity_rmse:.4f}")
    print("------------------")
    
    # Track initial performance
    rmse_history.append(([], baseline_authenticity_rmse))
    baseline_rmse = baseline_authenticity_rmse
    
    # Iterate over the sorted indices and if removing a feature map improves performance, keep it removed
    for idx, (channel_idx, importance_score) in enumerate(sorted_importance_scores):
        channel_idx = int(channel_idx)
        
        # Temporarily zero out this feature map
        layer.weight[channel_idx, ...] = 0
        if layer.bias is not None:
            layer.bias[channel_idx] = 0
        
        # Evaluate model with feature map removed
        predictions = get_predictions(model, dataloader, device)
        regression_errors = get_regression_errors(predictions)
        authenticity_pruned_rmse = get_rmse(regression_errors)
        
        print(f"Iteration {idx+1}/{len(sorted_importance_scores)}: " +
              f"Testing removal of channel {channel_idx}, " +
              f"Importance: {importance_score:.4f}, " +
              f"RMSE: {authenticity_pruned_rmse:.4f}")
        
        # Decide whether to keep this feature map removed
        if authenticity_pruned_rmse < baseline_rmse:
            baseline_rmse = authenticity_pruned_rmse # Update baseline RMSE
            removed_features.append(channel_idx)
            rmse_history.append((removed_features.copy(), baseline_rmse))
            print(f"  ✓ IMPROVING: Zeroing out feature map {channel_idx}")
        else:
            # Restore the feature map
            layer.weight[channel_idx, ...] = original_weights[channel_idx, ...]
            if layer.bias is not None:
                layer.bias[channel_idx] = original_bias[channel_idx]
            print(f"  ✗ NOT IMPROVING: Keeping feature map {channel_idx}")
        
        print(f"  Current best RMSE: {baseline_rmse:.4f}")
        print("------------------")
    
    # Final statistics
    print("\n------------------")
    print(f"Final RMSE: {baseline_rmse:.4f} after removing {len(removed_features)} feature maps")
    print(f"Improvement over baseline: {baseline_authenticity_rmse - baseline_rmse:.4f}")
    print(f"Feature reduction: {(len(removed_features)/len(sorted_importance_scores))*100:.1f}%")
    
    # Save the pruned model
    torch.save(model.state_dict(), model_path)
    
    return {
        'removed_features': removed_features,
        'baseline_rmse': baseline_authenticity_rmse,
        'final_rmse': baseline_rmse,
        'improvement': baseline_authenticity_rmse - baseline_rmse,
        'reduction_percentage': (len(removed_features)/len(sorted_importance_scores))*100,
        'rmse_history': rmse_history
    }

def remove_negative_impact_feature_maps(model, dataloader, device, layer_name, sorted_importance_scores, model_path='Models/negative_impact_pruned_model.pth'):
    """
    Remove feature maps that have a negative impact on model performance based on importance scores (impotance score > 0).
    
    Args:
        model: The neural network model
        dataloader: DataLoader for evaluation
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        sorted_importance_scores: List of tuples (channel_index, importance_score) sorted by importance
        
    Returns:
        Dictionary with pruning results and performance metrics
    """
    model.eval()
    model.to(device)
    
    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    
    # Create a backup of the original weights and bias
    original_weights = layer.weight.clone()
    original_bias = layer.bias.clone() if layer.bias is not None else None
    
    # Get baseline performance
    predictions = get_predictions(model, dataloader, device)
    regression_errors = get_regression_errors(predictions)
    authenticity_rmse = get_rmse(regression_errors)

    # Initialize tracking variables
    removed_features = []
    
    # Iterate over the sorted indices and zero out all the feature maps that have a negative impact (importance < 0)

    for idx, (channel_idx, importance_score) in enumerate(sorted_importance_scores):
        print(f"Iteration {idx} - Channel {channel_idx}: Importance score: {importance_score:.4f}")
        if importance_score > 0:
            channel_idx = int(channel_idx)
            layer.weight[channel_idx, ...] = 0
            if layer.bias is not None:
                layer.bias[channel_idx] = 0
            removed_features.append(channel_idx)

    # Evaluate model with feature maps removed
    new_predictions = get_predictions(model, dataloader, device)
    new_regression_errors = get_regression_errors(new_predictions)
    new_authenticity_rmse = get_rmse(new_regression_errors)

    # Save the pruned model
    torch.save(model.state_dict(), model_path)

    # Restore original weights for future use
    layer.weight.data.copy_(original_weights)
    if layer.bias is not None:
        layer.bias.data.copy_(original_bias)

    
    return {
        'removed_features': removed_features,
        'baseline_rmse': authenticity_rmse,
        'final_rmse': new_authenticity_rmse,
        'improvement': authenticity_rmse - new_authenticity_rmse,
        'reduction_percentage': (len(removed_features)/len(sorted_importance_scores))*100
    }

def remove_channels(model,device,layer_name,channels_indexes)->AuthenticityPredictor:
    """
    Remove channels, using an index list, from a convolutional layer in a model.
    
    Args:
        model: The neural network model
        device: Device to run the model on (cuda/cpu)
        layer_name: Name of the layer to optimize
        channels_indexes: List of channel indexes to remove
        
    Returns:
        The pruned model
    """
    model.eval()
    model.to(device)
    
    # Get the target layer
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
        
    # Zero out the specified channels
    for channel_idx in channels_indexes:
        layer.weight[channel_idx, ...] = 0
        if layer.bias is not None:
            layer.bias[channel_idx] = 0
    
    return model

# Creation of diffrent models using different pruning techniques

- Deletion of models is due to make sure that im not using the same model again and again (first draft, not sure if im correctlly restoring weights in each pruning technique)

In [33]:
# LAYER to prune
LAYER = 'features.28'
DEVICE = 'cuda'

# Base model for importance score computation
base_model = AuthenticityPredictor()
base_model.load_state_dict(torch.load('Models/VGG-16_authenticity_finetuned.pth'))
base_model.eval()
base_model.to(DEVICE)

sorted_importance_scores = compute_feature_map_importance(base_model, train_dataloader, DEVICE, LAYER)
np.save('Ranking_arrays/authenticity_importance_scores.npy', sorted_importance_scores)
del base_model

# Model for negative impact feature maps removal
negative_impact_model = AuthenticityPredictor()
negative_impact_model.load_state_dict(torch.load('Models/VGG-16_authenticity_finetuned.pth'))
negative_impact_model.eval()
negative_impact_model.to(DEVICE)

negative_impact_subset = remove_negative_impact_feature_maps(negative_impact_model, train_dataloader, DEVICE, LAYER, sorted_importance_scores, model_path='Models/authenticity_negative_impact_pruned_model.pth')

del negative_impact_model


# Model for noisy feature maps removal
noisy_pruning_model = AuthenticityPredictor()
noisy_pruning_model.load_state_dict(torch.load('Models/VGG-16_authenticity_finetuned.pth'))
noisy_pruning_model.eval()
noisy_pruning_model.to(DEVICE)

noisy_optimal_subset = remove_noisy_feature_maps(noisy_pruning_model, train_dataloader, DEVICE, LAYER, sorted_importance_scores, model_path='Models/authenticity_noise_out_pruned_model.pth')

del noisy_pruning_model



/tmp/ipykernel_1099599/762001622.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  base_model.load_state_dict(torch.load('Models/VGG-16_authenticity_finetuned.pth'))


Importance scores already computed, loading from file


/tmp/ipykernel_1099599/762001622.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  negative_impact_model.load_state_dict(torch.load('Models/VGG-16_authenticity_finetuned.

Iteration 0 - Channel 497.0: Importance score: 0.4390
Iteration 1 - Channel 423.0: Importance score: 0.1793
Iteration 2 - Channel 86.0: Importance score: 0.1675
Iteration 3 - Channel 458.0: Importance score: 0.1653
Iteration 4 - Channel 262.0: Importance score: 0.1554
Iteration 5 - Channel 113.0: Importance score: 0.1458
Iteration 6 - Channel 236.0: Importance score: 0.1354
Iteration 7 - Channel 47.0: Importance score: 0.1319
Iteration 8 - Channel 303.0: Importance score: 0.1291
Iteration 9 - Channel 165.0: Importance score: 0.1185
Iteration 10 - Channel 234.0: Importance score: 0.1174
Iteration 11 - Channel 137.0: Importance score: 0.1090
Iteration 12 - Channel 175.0: Importance score: 0.1079
Iteration 13 - Channel 224.0: Importance score: 0.1063
Iteration 14 - Channel 261.0: Importance score: 0.1062
Iteration 15 - Channel 326.0: Importance score: 0.1037
Iteration 16 - Channel 384.0: Importance score: 0.0995
Iteration 17 - Channel 213.0: Importance score: 0.0984
Iteration 18 - Channel

/tmp/ipykernel_1099599/762001622.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  noisy_pruning_model.load_state_dict(torch.load('Models/VGG-16_authenticity_finetuned.pt

Baseline authenticity RMSE: 11.0042
------------------
Iteration 1/512: Testing removal of channel 497, Importance: 0.4390, RMSE: 10.5652
  ✓ IMPROVING: Zeroing out feature map 497
  Current best RMSE: 10.5652
------------------
Iteration 2/512: Testing removal of channel 423, Importance: 0.1793, RMSE: 10.3927
  ✓ IMPROVING: Zeroing out feature map 423
  Current best RMSE: 10.3927
------------------
Iteration 3/512: Testing removal of channel 86, Importance: 0.1675, RMSE: 10.2352
  ✓ IMPROVING: Zeroing out feature map 86
  Current best RMSE: 10.2352
------------------
Iteration 4/512: Testing removal of channel 458, Importance: 0.1653, RMSE: 10.0939
  ✓ IMPROVING: Zeroing out feature map 458
  Current best RMSE: 10.0939
------------------
Iteration 5/512: Testing removal of channel 262, Importance: 0.1554, RMSE: 9.9633
  ✓ IMPROVING: Zeroing out feature map 262
  Current best RMSE: 9.9633
------------------
Iteration 6/512: Testing removal of channel 113, Importance: 0.1458, RMSE: 9.83

# Testing with already saved weights

In [9]:
BASE_MODEL = 'Models/VGG-16_authenticity_finetuned.pth'
NEGATIVE_IMPACT_PRUNED_MODEL_PATH = 'Models/authenticity_negative_impact_pruned_model.pth'

base_model = AuthenticityPredictor()
base_model.load_state_dict(torch.load(BASE_MODEL, weights_only=True))

negative_impact_pruned_model = AuthenticityPredictor()
negative_impact_pruned_model.load_state_dict(torch.load(NEGATIVE_IMPACT_PRUNED_MODEL_PATH,weights_only=True))

noisy_optimal_subset = AuthenticityPredictor()
noisy_optimal_subset.load_state_dict(torch.load('Models/authenticity_noise_out_pruned_model.pth',weights_only=True))
# Testing

# Test the baseline model
print("Testing the baseline model")
test_model(base_model, test_dataloader, criterion, device)
print("------------------")

# test the negative impact pruned model
print("Testing the negative impact pruned model")
test_model(negative_impact_pruned_model, test_dataloader, criterion, device)
print("------------------")

# test the noisy optimal subset pruned model
print("Testing the noisy optimal subset pruned model")
test_model(noisy_optimal_subset, test_dataloader, criterion, device)
print("------------------")


Testing the baseline model
Test Loss: 152.4362
------------------
Testing the negative impact pruned model
Test Loss: 120.3955
------------------
Testing the noisy optimal subset pruned model
Test Loss: 90.7545
------------------


# Final Aanalysis - Comparing the models zeroed out weights & Correlations between the models predicitons and ground truth

## Zero-out weights analysis

In [ ]:
# Create a function that extract the indices of the zeroed out feature maps in a convolutional layer

def get_zeroed_feature_maps(model, layer_name):
    """
    Get the indices of the zeroed out feature maps in a convolutional layer.

    Args:
        model (nn.Module): The neural network model.
        layer_name (str): The name of the convolutional layer.

    Returns:
        list: The indices of the zeroed out feature maps.
    """
    dict_modules = dict(model.named_modules())
    layer = dict_modules[layer_name]
    zeroed_feature_maps = []

    for i, weight in enumerate(layer.weight):
        if torch.all(weight == 0):
            zeroed_feature_maps.append(i)
    zeroed_feature_maps.sort()

    num_zeroed = len(zeroed_feature_maps)

    return zeroed_feature_maps, num_zeroed

# Get the zeroed out feature maps in the 'features.28' layer of the noisy pruned model
_, noisy_num_zeroed = get_zeroed_feature_maps(noisy_pruned_model, 'features.28')

# Get the zeroed out feature maps in the 'features.28' layer of the best subset pruned model
_, best_subset_num_zeroed = get_zeroed_feature_maps(best_subset_pruned_model, 'features.28')

# Get the zeroed out feature maps in the 'features.28' layer of the negative impact pruned model
_, negative_impact_num_zeroed = get_zeroed_feature_maps(negative_impact_pruned_model, 'features.28')

print(f"Noisy pruned model: {noisy_num_zeroed} zeroed out feature maps")
print(f"Best subset pruned model: {best_subset_num_zeroed} zeroed out feature maps")
print(f"Negative impact pruned model: {negative_impact_num_zeroed} zeroed out feature maps")


# Test RSA on the models (baseline and pruned models)

### Definition of the models for RSA

In [13]:
BASELINE_MODEL_NAME = 'Models/VGG-16_authenticity_finetuned.pth'
NOISY_PRUNED_MODEL_PATH = 'Models/authenticity_noise_out_pruned_model.pth'
NEGATIVE_IMPACT_PRUNED_MODEL_PATH = 'Models/authenticity_negative_impact_pruned_model.pth'

baseline_model = AuthenticityPredictor()
baseline_model.load_state_dict(torch.load(BASELINE_MODEL_NAME, weights_only=True))

noisy_pruned_model = AuthenticityPredictor()
noisy_pruned_model.load_state_dict(torch.load(NOISY_PRUNED_MODEL_PATH, weights_only=True))

negative_impact_pruned_model = AuthenticityPredictor()
negative_impact_pruned_model.load_state_dict(torch.load(NEGATIVE_IMPACT_PRUNED_MODEL_PATH,weights_only=True))

<All keys matched successfully>

## Helper functions for RSA

In [14]:
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity

In [15]:

class FeatureMapHook:
    """Hook to extract feature maps from neural network layers."""
    
    def __init__(self):
        self.feature_maps = []
    
    def __call__(self, module, input, output):
        # Detach from computation graph and move to CPU
        self.feature_maps.append(output.detach().cpu())

def get_feature_maps(model, dataloader, layer_name, device):
    """
    Extracts the feature maps of a specific layer from a model.
    
    Args:
        model (nn.Module): The neural network model.
        dataloader (DataLoader): DataLoader for evaluation.
        layer_name (str): The name of the layer to extract feature maps from.
        device (str): Device to run the model on ('cuda' or 'cpu').
        
    Returns:
        np.ndarray: The feature maps as a numpy array with shape (240, num_features).
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)
    
    # Register a hook to extract feature maps
    hook = FeatureMapHook()
    target_layer = dict(model.named_modules())[layer_name]
    hook_handle = target_layer.register_forward_hook(hook)
    
    # Forward pass to extract feature maps from the dataloader
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            model(inputs)

    # Remove the hook
    hook_handle.remove()
    
    # Process the feature maps to get the desired shape
    all_features = []
    
    for batch_features in hook.feature_maps:
        # Handle different possible output formats (accommodate different layer types)
        if len(batch_features.shape) == 4:  # Conv layers: [batch_size, channels, height, width]
            batch_size, channels, height, width = batch_features.shape
            # Flatten spatial dimensions and create one feature vector per sample
            batch_features = batch_features.reshape(batch_size, channels * height * width)
        elif len(batch_features.shape) == 2:  # Linear layers: [batch_size, features]
            pass  # Already in the right format
        
        # Add batch features to our collection
        all_features.append(batch_features)
    
    # Concatenate all batches and convert to numpy
    features_tensor = torch.cat(all_features, dim=0)
    
    # Ensure we have exactly the number of samples we expect in the dataloader 
    assert features_tensor.shape[0] == len(dataloader.dataset) 
    
    # Convert to numpy array
    features_array = features_tensor.numpy()
    
    return features_array
def compute_similarity_matrix(features):
    """
    Compute a similarity matrix from feature embeddings.
    Works with both convolutional features (4D) and FC features (2D).
    
    Args:
        features: numpy array - either shape (n_samples, n_channels, height, width)
                 or shape (n_samples, n_features)
        
    Returns:
        similarity_matrix: numpy array of shape (n_samples, n_samples)
    """
    # Check the dimensionality of features
    n_samples = features.shape[0]
    
    # If features are from convolutional layer (4D), reshape to 2D
    if len(features.shape) == 4:
        features_flat = features.reshape(n_samples, -1)
    else:
        # Features are already 2D (from FC layer)
        features_flat = features
    
    # Compute cosine similarity between all pairs
    similarity_matrix = cosine_similarity(features_flat)
    
    return similarity_matrix

def compute_quality_difference_matrix(quality_scores):
    """
    Compute a matrix of quality differences between all pairs of samples.
    
    Args:
        quality_scores: numpy array of shape (n_samples,) containing quality scores
        
    Returns:
        difference_matrix: numpy array of shape (n_samples, n_samples)
    """
    n_samples = quality_scores.shape[0]
    difference_matrix = np.zeros((n_samples, n_samples))
    
    # Compute absolute differences between all pairs
    for i in range(n_samples):
        for j in range(n_samples):
            difference_matrix[i, j] = abs(quality_scores[i] - quality_scores[j])
            
    return difference_matrix

def get_upper_triangle(matrix):
    """
    Extract the upper triangle of a matrix (excluding diagonal).
    
    Args:
        matrix: numpy array of shape (n, n)
        
    Returns:
        upper_triangle: flattened upper triangle values
    """
    indices = np.triu_indices_from(matrix, k=1)
    return matrix[indices]

def calculate_fit(similarity_matrix, quality_diff_matrix):
    """
    Calculate the fit between similarity and quality difference matrices.
    
    Args:
        similarity_matrix: numpy array of shape (n_samples, n_samples)
        quality_diff_matrix: numpy array of shape (n_samples, n_samples)
        
    Returns:
        correlation: Spearman correlation coefficient between the matrices
        p_value: p-value of the correlation
    """
    # Extract upper triangles (excluding diagonal)
    sim_upper = get_upper_triangle(similarity_matrix)
    qual_upper = get_upper_triangle(quality_diff_matrix)
    
    # Compute correlation (negative since higher similarity should correspond to lower difference)
    correlation, p_value = spearmanr(sim_upper, qual_upper)
    
    # We're expecting a negative correlation (higher similarity → lower quality difference)
    # so we return the negative correlation value for easier interpretation
    return -correlation, p_value


In [16]:
# Get feature maps
baseline_features = get_feature_maps(baseline_model, test_dataloader, 'fc1.3', device)

noisy_pruned_features = get_feature_maps(noisy_pruned_model, test_dataloader, 'fc1.3', device)

negative_impact_features = get_feature_maps(negative_impact_pruned_model, test_dataloader, 'fc1.3', device)

# Extract quality scores
auth_scores_list = []
with torch.no_grad():
	for _, labels in test_dataloader:
		auth_scores_list.append(labels[:, 0])  # First column contains auth scores
q_scores = torch.cat(auth_scores_list).numpy()




In [18]:
# Compute similarity matrices
baseline_similarity = compute_similarity_matrix(baseline_features)
noisy_similarity = compute_similarity_matrix(noisy_pruned_features)
negative_impact_similarity = compute_similarity_matrix(negative_impact_features)

# Compute quality difference matrices
quality_diff_matrix = compute_quality_difference_matrix(q_scores)

# Calculate fit between similarity and quality difference matrices
baseline_fit = calculate_fit(baseline_similarity, quality_diff_matrix)
noisy_fit = calculate_fit(noisy_similarity, quality_diff_matrix)
negative_impact_fit = calculate_fit(negative_impact_similarity, quality_diff_matrix)

print("Baseline Model Fit:")
print(f"Correlation: {baseline_fit[0]:.4f}")
print("------------------")
print("RSME Noise-out Pruned Model Fit:")
print(f"Correlation: {noisy_fit[0]:.4f}")
print("------------------")
print("RSME Negative Impact Pruned Model Fit:")
print(f"Correlation: {negative_impact_fit[0]:.4f}")


Baseline Model Fit:
Correlation: 0.0139
------------------
RSME Noise-out Pruned Model Fit:
Correlation: 0.0070
------------------
RSME Negative Impact Pruned Model Fit:
Correlation: 0.0141
